In [15]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub
import numpy as np
    
import pandas as pd
from sklearn.model_selection import train_test_split

### Data taken from CFPB, the data when a consumer files a grievance against a bank. The target variable is the product column here.

In [16]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [17]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [18]:
pd.set_option('display.max_colwidth', -1)
df

D:\Anaconda3\envs\tf-keras-gpu-test\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter 

In [19]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

### We can see here that the first 2 classes have high numbers but the others are not too representative. Implying the dataset is unbalanced.

In [21]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight(
    class_weight = 'balanced',
    classes = np.unique(df['Product']),
    y = df['Product']))  
#here we assign a higher weight to a class that is less represented and lower weight to a class that is more

In [22]:
class_weights.sort()

In [23]:
class_weights

[0.43980801028844385,
 0.7258015614340938,
 0.9771915501581794,
 1.3673425674710837,
 3.2459322033898306,
 3.4998172514619883]

In [24]:
weights={}
for index, weight in enumerate(class_weights) :
    weights[index]=weight

In [25]:
weights  #as expected the weigts for the frequent classes are less

{0: 0.43980801028844385,
 1: 0.7258015614340938,
 2: 0.9771915501581794,
 3: 1.3673425674710837,
 4: 3.2459322033898306,
 5: 3.4998172514619883}

### We would use the 'Product' column as the target and the 'Consumer complaint narrative' as the feature text

In [26]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train['Consumer complaint narrative'].values, X_train['Product'].values))
dataset_test = tf.data.Dataset.from_tensor_slices((X_test['Consumer complaint narrative'].values, X_test['Product'].values))

In [29]:
for text, target in dataset_train.take(5):
    print('Complaint: {}, Target: {}'.format(text, target))

Complaint: b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.', Target: b'Debt collection'
Complaint: b'I signed up for Citibank Checking and Savings account promotion program. Nowhere in their promotion requirements does it say that it does not apply to Texas. You can see their promotion published here, all it said was that " To enroll in this offer you must apply directly from this page. Citi rates and promotional offers may vary by state. Looks like you\'re in Texas. \'\' I even re-selected the state to make sure it does not tell me that Texas is not eligible, but it didn\'t say anything. Also in their fine prints they did not list the states that were eligible so who would know if they don\'t qualify? This is scam by misleading people in believing they will would qualify so they will open an account with them. See the advertised deal here, you can see nothing about the states. \n\nhttps : //b

In [30]:
for text, target in dataset_test.take(5):
    print('Complaint: {}, Target: {}'.format(text, target))

Complaint: b"On XX/XX/XXXX, I paid off the account in full to the original creditor. During this phone call, the original creditor informed me that they will close the account with the debt collection agency. \n\nOn XX/XX/XXXX, I called the original creditor regarding the account closure because I will still seeing the debt on my credit report. During this phone call, they said they had emailed the supervisor of the collection agency on XX/XX/XXXX to close the account and that the collection account will be closed soon. \n\nOn XX/XX/XXXX, I called the supervisor of the collection agency who informed me that he never received any communication from the original creditor regarding the closure of this account. He informed me that he needed a formal confirmation from the original creditor in order to close the account. I placed the supervisor of the collection agency on a 3-way phone call with the original creditor and myself. During this phone call, the supervisor told the original credit

### Converting the target to numerical representation of classes

In [31]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(['Debt collection', 'Credit card or prepaid card', 'Mortgage', 'Checking or savings account', 'Student loan', 'Vehicle loan or lease']),
        values=tf.constant([0, 1, 2, 3, 4, 5]),
    ),
    default_value=tf.constant(-1),
    name="target_encoding"
)

@tf.function
def target(x):
    return table.lookup(x)

In [32]:
#printing the feateure text and converted target
def show_batch(dataset, size=5):
    for batch, label in dataset.take(size):
        print(batch.numpy())
        print(target(label).numpy())

In [33]:
show_batch(dataset_test,6)

b"On XX/XX/XXXX, I paid off the account in full to the original creditor. During this phone call, the original creditor informed me that they will close the account with the debt collection agency. \n\nOn XX/XX/XXXX, I called the original creditor regarding the account closure because I will still seeing the debt on my credit report. During this phone call, they said they had emailed the supervisor of the collection agency on XX/XX/XXXX to close the account and that the collection account will be closed soon. \n\nOn XX/XX/XXXX, I called the supervisor of the collection agency who informed me that he never received any communication from the original creditor regarding the closure of this account. He informed me that he needed a formal confirmation from the original creditor in order to close the account. I placed the supervisor of the collection agency on a 3-way phone call with the original creditor and myself. During this phone call, the supervisor told the original creditor that no 

In [34]:
#function to apply the one hot encoding to the classes
def fetch(text, labels):
    return text, tf.one_hot(target(labels),6)

In [35]:
train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [36]:
next(iter(train_data_f))

(<tf.Tensor: shape=(), dtype=string, numpy=b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.'>,
 <tf.Tensor: shape=(6,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0.], dtype=float32)>)

In [37]:
train_data, train_labels = next(iter(train_data_f.batch(5)))
train_data, train_labels

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.',
        b'I signed up for Citibank Checking and Savings account promotion program. Nowhere in their promotion requirements does it say that it does not apply to Texas. You can see their promotion published here, all it said was that " To enroll in this offer you must apply directly from this page. Citi rates and promotional offers may vary by state. Looks like you\'re in Texas. \'\' I even re-selected the state to make sure it does not tell me that Texas is not eligible, but it didn\'t say anything. Also in their fine prints they did not list the states that were eligible so who would know if they don\'t qualify? This is scam by misleading people in believing they will would qualify so they will open an account with them. See the advertised deal here, you can see nothing about the states. \n

### Building the model

In [38]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1" #pretrained embedding from tfhub trained on google news of dimension 128
hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 0.33617228,  0.23203762, -0.09538511, -0.06548779, -0.08765431,
        -0.01077191, -0.00245969,  0.05258209, -0.06241624,  0.19836168,
         0.09080565, -0.12921023,  0.0396726 , -0.05281024,  0.00817845,
         0.01611447, -0.17562988,  0.07727271, -0.19410342,  0.28781003,
        -0.18966146,  0.22917116,  0.02318892, -0.05648421,  0.0852662 ,
        -0.15905663,  0.12677582, -0.08696103, -0.10879584,  0.0401351 ,
         0.04865107, -0.06219418,  0.01057932, -0.09699482, -0.10901408,
         0.12662935, -0.00856555, -0.17180865, -0.16313195,  0.02499727,
         0.05349051, -0.18023014,  0.03826252,  0.04737706,  0.06879856,
         0.07156485, -0.10703883,  0.02409267, -0.02078929,  0.06535412,
        -0.05628244,  0.12537208,  0.01271748, -0.00653138,  0.02049751,
        -0.03821641, -0.05212895, -0.0351408 , -0.11024246,  0.04053358,
        -0.07984847,  0.00559551, -0.07771305, -0.0565149 ,  0.10259037,
 

In [40]:
train_data[:1]

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.'],
      dtype=object)>

In [41]:
model = tf.keras.Sequential()
model.add(hub_layer)
for units in [128, 128, 64 , 32]:                #for loop for testing out different models for easy editing
    model.add(tf.keras.layers.Dense(units, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(6, activation='softmax'))     #6 class softmax

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0

In [42]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
train_data_f=train_data_f.shuffle(70000).batch(512)    #creating batched of the training data and shuffling the data
test_data_f=test_data_f.batch(512)